# Process data

## 1. Setup

In [ ]:
import ast
import json
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset

## 2. Process

In [ ]:
data = load_dataset('Universal-NER/Pile-NER-type')['train']

In [ ]:
prep = []
for d in tqdm(data):
    utters = d['conversations']

    entities = []
    for uid, u in enumerate(utters):
        if uid == 0:
            text = u['value'][6:]
        
        elif uid == 1:
            continue

        elif uid % 2 == 0:
            entity_type = u['value'].replace('What describes ', '').replace(' in the text?', '')

        else:
            entity_mentions = ast.literal_eval(u['value'])
            entities.append({'entity_type': entity_type, 'entity_mentions': entity_mentions})
            
    prep.append({'text': text, 'entities': entities})

In [ ]:
with open('data.json', 'w') as f:
    json.dump(prep, f)

## 3. Push to Hugginface Hub

In [ ]:
data = Dataset.from_json('data.json')

In [ ]:
data = data.train_test_split(test_size=0.2)
train_data, valid_data = data['train'], data['test']
len(train_data), len(valid_data)

In [ ]:
train_data.push_to_hub('yongsun-yoon/open-ner-english', split='train')
valid_data.push_to_hub('yongsun-yoon/open-ner-english', split='validation')